# Palabras clave (1ª versión)

## Frecuencia de aparición

In [1]:
from collections import defaultdict

def get_keywords(words):
    word_counts = defaultdict(lambda : 0)
    for w in words:
        word_counts[w]+= 1
    return [ x for x,_ in sorted(word_counts.iteritems(), key=lambda (_,freq):freq, reverse=True)][:10]

## Carga de documentos

Dataset URL: http://www.lsi.us.es/~fermin/corpusCine.zip

Nos quedamos con los ficheros .xml, que tienen una estructura del estilo:

```xml
<review author="XXX" title="XXX" rank="X" maxRank="X" source="XXX">
	<summary>XXX</summary>
	<body>XXX</body>
</review>
```

In [2]:
import os
import config

### Detección de la codificación

In [3]:
import chardet

with open(config.DATASET_MUCHOCINE_RAW+'/100.xml', 'r') as fd:
    txt = fd.read()
    
print chardet.detect(txt)

{'confidence': 0.9248548221453007, 'encoding': 'ISO-8859-2'}


### Parseado con expresiones regulares

In [4]:
import codecs  # Know your encoding
import re

def parse_file(file_path):
    retval = {'file': os.path.basename(file_path)}
    with codecs.open(file_path,'r', encoding='ISO-8859-2') as fd:
        f = fd.read().strip()
    reg_expr = re.compile(r'\<review author="(?P<author>.*)" title="(?P<title>.*)" rank="(?P<rank>\d)".*\>\s*<summary>(?P<summary>.*)</summary>\s*<body>(?P<body>.*)</body>\s*</review>')
    regexp_result = reg_expr.search(f.strip())
    for key in ['author', 'rank', 'title', 'summary', 'body']:
        retval[key] = regexp_result.group(key)
    return retval

### Carga

In [5]:
documents = []
for file_name in os.listdir(config.DATASET_MUCHOCINE_RAW):
    try:
        documents.append(
            parse_file(config.DATASET_MUCHOCINE_RAW+'/'+file_name)
        )
    except Exception as e:
        print file_name
        print e.message
documents.sort(key=lambda x: x['file'])

## Documento de ejemplo

In [6]:
def get_id_by_title(title):
    result = [(i, documents[i]['file']) for i in range(len(documents)) if title==documents[i]['title']]
    return result
my_id = get_id_by_title('Harry Potter y la piedra filosofal')[-1][0]

In [7]:
print documents[my_id]


{'body': u'Antes de nada, me declaro fan incondicional de los libros de Harry y Potter. De sus pel\xedculas soy un poco menos fan, aunque me he comprado el pack de DVDs con las tres primeras. Y que verg\xfcenza al comprarlas, que me dijeron que esas las vend\xeda s\xf3lo en la secci\xf3n de ni\u0144os, que no eran pel\xedculas de adultos. Pues ale, me alegro, que adem\xe1s me salieron m\xe1s baratas que las de Star Wars.La pel\xedcula trata sobre un chaval que es dejado en una casa por un grupo de extra\u0144os magos cuando no es m\xe1s que un bebe. Doce a\u0144os m\xe1s tarde, el chaval ha crecido y su situaci\xf3n en la vida ha empeorado considerablemente. Vive debajo de la escalera en la casa de sus t\xedos, los cuales le odian y es vejado continuamente por su desagradable primo. Pero esta situaci\xf3n que tiene cambia el d\xeda que recibe una extra\u0144a carta de un colegio para magos.La primera de la saga de Harry Potter nos presenta a los personajes de la historia de un modo pau

## 1ª versión. Contar palabras

In [8]:
def txt2words1(txt):
    words = [w for w in txt.split(' ') if w!='']
    return words

In [9]:
print get_keywords(txt2words1(documents[my_id]['body']))

[u'de', u'que', u'la', u'me', u'un', u'el', u'los', u'no', u'es', u'y']


## 2ª versión. Stopwords

In [10]:
## Taken from NLTK corpus so you don't have to download

#import nltk
#nltk.download()
#from nltk.corpus import stopwords
#print filter_symbols(
#    ' '.join(
#        sorted(stopwords.words('spanish'))
#    )
#)

STOPWORDS = set('''
a al algo algunas algunos ante antes como con contra cual cuando de del desde donde durante e el ella ellas ellos
en entre era erais eran eras eres es esa esas ese eso esos esta estaba estabais estaban estabas estad estada estadas
estado estados estamos estando estar estaremos estara estaran estaras estare estareis estaria estariais estariamos
estarian estarias estas este estemos esto estos estoy estuve estuviera estuvierais estuvieran estuvieras estuvieron
estuviese estuvieseis estuviesen estuvieses estuvimos estuviste estuvisteis estuvieramos estuviesemos estuvo esta
estabamos estais estan estas este esteis esten estes fue fuera fuerais fueran fueras fueron fuese fueseis fuesen fueses
fui fuimos fuiste fuisteis fueramos fuesemos ha habida habidas habido habidos habiendo habremos habra habran habras
habre habreis habria habriais habriamos habrian habrias habeis habia habiais habiamos habian habias han has hasta
hay haya hayamos hayan hayas hayais he hemos hube hubiera hubierais hubieran hubieras hubieron hubiese hubieseis
hubiesen hubieses hubimos hubiste hubisteis hubieramos hubiesemos hubo la las le les lo los me mi mis mucho muchos
muy mas mi mia mias mio mios nada ni no nos nosotras nosotros nuestra nuestras nuestro nuestros o os otra otras otro
otros para pero poco por porque que quien quienes que se sea seamos sean seas sentid sentida sentidas sentido sentidos
seremos sera seran seras sere sereis seria seriais seriamos serian serias seais siente sin sintiendo sobre sois somos
son soy su sus suya suyas suyo suyos si tambien tanto te tendremos tendra tendran tendras tendre tendreis tendria
tendriais tendriamos tendrian tendrias tened tenemos tenga tengamos tengan tengas tengo tengais tenida tenidas tenido
tenidos teniendo teneis tenia teniais teniamos tenian tenias ti tiene tienen tienes todo todos tu tus tuve tuviera
tuvierais tuvieran tuvieras tuvieron tuviese tuvieseis tuviesen tuvieses tuvimos tuviste tuvisteis tuvieramos tuviesemos
tuvo tuya tuyas tuyo tuyos tu un una uno unos vosostras vosostros vuestra vuestras vuestro vuestros y ya yo el eramos
'''.split())

def txt2words2(txt):
    words = [w for w in txt.split(' ') if w!='' and w not in STOPWORDS]
    return words

In [11]:
print get_keywords(txt2words2(documents[my_id]['body']))

[u'm\xe1s', u'pel\xedculas', u'chaval', u'pel\xedcula', u'personajes', u'ser', u'Harry', u'casa', u'situaci\xf3n', u'infantiles.']


## 3ª versión. Quitando mayúsculas, tildes...

In [12]:
def txt2words3(txt):
    txt = txt.lower()  # Text in lowercase
    table = dict(zip( #  Quitar tildes
        [ord(x) for x in u'áéíóúü'],
        [ord(x) for x in u'aeiouu']
    ))
    txt = txt.translate(table)    
    txt = ''.join([
        letter for letter in txt 
        if letter in set(u'abcdefghijklmnñopqrstuvwxyz0123456789 ')]
    )
    words = [w for w in txt.split(' ') if w!='' and w not in STOPWORDS]
    return words

In [13]:
print get_keywords(txt2words3(documents[my_id]['body']))

[u'pelicula', u'fan', u'casa', u'harry', u'chaval', u'luego', u'adultos', u'peliculas', u'personajes', u'potter']


## 4ª versión. Raíces léxicas

In [16]:
from nltk.stem.snowball import SpanishStemmer
stemmer = SpanishStemmer()

def txt2words4(txt):
    txt = txt.lower()  # Text in lowercase
    table = dict(zip( #  Quitar tildes
        [ord(x) for x in u'áéíóúü'],
        [ord(x) for x in u'aeiouu']
    ))
    txt = txt.translate(table)    
    txt = ''.join([
        letter for letter in txt 
        if letter in set(u'abcdefghijklmnñopqrstuvwxyz0123456789 ')]
    )
    words = [
        stemmer.stem(w)
        for w in txt.split(' ')
        if w!='' and w not in STOPWORDS
    ]
    return words

In [17]:
print get_keywords(txt2words4(documents[my_id]['body']))

[u'pelicul', u'gust', u'primer', u'personaj', u'encant', u'fan', u'magic', u'cas', u'cre', u'harry']


## Aplicándolo a todos los documentos

In [18]:
for d in documents:
    for key in ['body', 'summary']:
        d[key+'_tokens'] = txt2words4(d[key])

### Exploración

In [19]:
# Film titles
titles = sorted(list({d['title'] + ':' + d['file'] for d in documents}))
print '\n'.join(titles)

13 tzameti:1212.xml
13 tzameti:1288.xml
13 tzameti:1299.xml
13 tzameti:1372.xml
13 tzameti:1374.xml
13 tzameti:624.xml
14 Kilómetros:3636.xml
1408:3163.xml
1408:3182.xml
1408:3672.xml
1408:3691.xml
1408:3697.xml
1408:3717.xml
1408:3752.xml
1408:3770.xml
1408:3791.xml
1408:3793.xml
1408:3796.xml
1408:3878.xml
1408:3904.xml
1408:3948.xml
15 días contigo:1059.xml
16 calles:153.xml
16 calles:2776.xml
187:3687.xml
2 Días en París:2640.xml
20.000 ańos en Sing Sing:1706.xml
2001, Odisea del espacio:1922.xml
28 semanas después:2356.xml
28 semanas después:2364.xml
28 semanas después:2368.xml
28 semanas después:2372.xml
28 semanas después:2379.xml
28 semanas después:2380.xml
28 semanas después:2393.xml
28 semanas después:2395.xml
28 semanas después:2400.xml
28 semanas después:2423.xml
28 semanas después:2476.xml
28 semanas después:2533.xml
28 semanas después:2608.xml
28 semanas después:2651.xml
28 semanas después:2656.xml
28 semanas después:2768.xml
30 días de oscuridad:3597.xml
30 días de oscur

In [22]:
def explore(title):
    print title
    for kws in [get_keywords(d['body_tokens']) for d in documents if title==d['title']]:
        print ', '.join(kws)

In [23]:
explore(u'Harry Potter y la piedra filosofal')

Harry Potter y la piedra filosofal
pelicul, mag, demasi, primer, ser, nios, personaj, termin, represent, libr
pelicul, gust, primer, personaj, encant, fan, magic, cas, cre, harry


In [24]:
explore('High School Musical')

High School Musical
pelicul, disc, nunc, gust, disney, lad, meditant, ejem, ocasion, ver
mejor, original, pelicul, school, coreografi, disney, result, juvenil, sobresalient, produccion


In [25]:
explore('Los puentes de Madison')

Los puentes de Madison
amor, ser, famili, human, cas, ultim, pelicul, vid, valor, verdader


In [26]:
explore('2001, Odisea del espacio')

2001, Odisea del espacio
2001, ser, final, film, pelicul, kubrick, propi, director, llam, tan


In [27]:
explore('Alien vs Predator 2')

Alien vs Predator 2
men, super, especial, rod, enfrent, falt, nuev, ser, pues, efect
predator, pelicul, original, ali, human, ver, guion, accion, cualqui, muert
ali, nunc, pelicul, demasi, predator, cutr, hech, mejor, mal, patet
predator, aliens, pelicul, ali, entreg, bastant, ident, vs, segund, requiem
pelicul, parec, mism, sal, ali, videojueg, sol, depred, aport, straus
histori, hac, version, personaj, pelicul, baj, ser, result, hech, part
pelicul, depred, ali, histori, qued, aliens, pas, medi, ahi, nav
pelicul, primer, part, dos, exact, cas, despu, aliens, depred, tod
casi, bien, carg, tont, dos, primer, ver, secuel, segund, luch
ali, pelicul, nunc, predator, pues, tierr, sal, mal, patet, human


In [28]:
explore('Apocalypto')

Apocalypto
pelicul, may, nativ, gran, asi, necesari, actor, distint, principal, vez
ser, apocalypt, pelicul, gibson, may, acab, caz, persecucion, guerrer, propi
gibson, asi, histori, pelicul, mism, cuent, director, mel, jagu, cin
pelicul, histori, plan, dich, mund, gibson, aventur, estil, apocalypt, represent
violent, bien, peli, civilizacion, histor, desarroll, suficient, persecucion, rigur, ocup
son, pelicul, nuev, men, pantall, entiend, fin, histori, anterior, cont
gibson, tan, pelicul, pued, mel, jagu, salvaj, max, apocalypt, enfrent
gibson, primer, sol, parec, cin, aztec, human, pelicul, mund, realid
may, pelicul, hac, alde, pas, siend, destru, trat, grab, bien
mel, hac, indigen, consegu, interpret, cuent, sol, direccion, vist, pelicul
gust, apocalypt, cin, histori, gibson, cuent, pelicul, ultim, mejor, prim
human, cabez, tip, esper, aun, cre, vision, sub, gener, met
pelicul, cin, director, apocalypt, gibson, ser, crist, pasion, qued, autor
vez, alde, mod, accion, dic, peli, pued,

## Exportar/Importar

In [29]:
import json
import zlib


# Export
with open(config.DATASET_MUCHOCINE, 'w+') as fd:
    fd.write(zlib.compress(json.dumps(documents)))
    
# Import
with open(config.DATASET_MUCHOCINE, 'r') as fd:
    docs_import =json.loads(zlib.decompress(fd.read()))